In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [2]:
# Function to Preprocess the data
def preprocess_images(image_dir, image_size=(64, 64)):
    images = []
    labels = []
    label_names = os.listdir(image_dir)
    label_dict = {label: idx for idx, label in enumerate(label_names)}
    
    for label in label_names:
        image_files = os.listdir(os.path.join(image_dir, label))
        for image_file in image_files:
            image_path = os.path.join(image_dir, label, image_file)
            image = cv2.imread(image_path)
            image = cv2.resize(image, image_size)
            images.append(image)
            labels.append(label_dict[label])
    
    images = np.array(images) / 255.0
    labels = np.array(labels)
    return images, labels, label_dict

In [3]:
# Function to Create and Train the model

def create_cnn_model(input_shape, num_classes):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

def train_model(X_train, y_train, input_shape, num_classes):
    model = create_cnn_model(input_shape, num_classes)
    model.fit(X_train, y_train, epochs=20, validation_split=0.2, batch_size=32)
    model.save('gesture_model.h5')
    return model

In [4]:
# Function to Evaluate the Model
def evaluate_model(model, X_test, y_test):
    test_loss, test_accuracy = model.evaluate(X_test, y_test)
    print(f'Test accuracy: {test_accuracy}')

In [5]:
# Function to  Predict the Gestures
def predict_gesture(model, image_path, label_dict):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (64, 64))
    image = np.expand_dims(image, axis=0) / 255.0
    prediction = model.predict(image)
    return np.argmax(prediction), label_dict

In [30]:
# Set the image directory
image_dir = 'C:\\Partition_non_sys\\Programming files\\Python files\\archive\\leapGestRecog\\01'

In [31]:
# Preprocess the data
print("Preprocessing the data...")
images, labels, label_dict = preprocess_images(image_dir)
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

Preprocessing the data...


In [32]:
# Train the model
print("Training the model...")
input_shape = (64, 64, 3)
num_classes = len(label_dict)
model = train_model(X_train, y_train, input_shape, num_classes)

Training the model...
Epoch 1/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 4s 34ms/step - accuracy: 0.5409 - loss: 1.3855 - val_accuracy: 1.0000 - val_loss: 0.0058
Epoch 2/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.9844 - loss: 0.0562 - val_accuracy: 1.0000 - val_loss: 2.0246e-04
Epoch 3/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.9907 - loss: 0.0239 - val_accuracy: 1.0000 - val_loss: 7.2728e-05
Epoch 4/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.9956 - loss: 0.0149 - val_accuracy: 1.0000 - val_loss: 1.0609e-04
Epoch 5/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.9955 - loss: 0.0145 - val_accuracy: 1.0000 - val_loss: 7.0493e-05
Epoch 6/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.9982 - loss: 0.0060 - val_accuracy: 1.0000 - val_loss: 3.6077e-05
Epoch 7/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.9980 - loss: 0.0059 - val_accuracy: 1.0000 - val_loss: 4.2432e-06
Epoch 8/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.

In [33]:
# Evaluate the model
print("Evaluating the model...")
evaluate_model(model, X_test, y_test)

Evaluating the model...
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 2.6666e-08
Test accuracy: 1.0


In [49]:
# Predict a gesture
print("Predicting a gesture...")
image_path = 'path/image.jpg'  # Replace with the path to your image
labels = ['Palm','L','Fist','Fist Moved','Thumb','Index','OK','Palm Moved','C','Down']
predicted_label, _ = predict_gesture(model, image_path, {v: k for k, v in label_dict.items()})
print(f'Predicted gesture: {labels[predicted_label]}')

Predicting a gesture...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted gesture: Down
